<a href="https://colab.research.google.com/github/deivisongithub/OCR_for_numbers/blob/main/OCR_Guaraves_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Escaneamento de imagem do painel Guaraves (UFPB)

# Importando as bibliotecas

In [ ]:
import numpy as np
import cv2
import imutils
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import csv

# Capturando tela (deverá ser a partir da camera)



In [ ]:
!wget https://i.postimg.cc/m2y9Gnzw/guaraves.jpg ./

--2021-12-28 20:08:16--  https://i.postimg.cc/m2y9Gnzw/guaraves.jpg
Resolving i.postimg.cc (i.postimg.cc)... 104.238.220.13
Connecting to i.postimg.cc (i.postimg.cc)|104.238.220.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169356 (165K) [image/jpeg]
Saving to: ‘guaraves.jpg.4’

guaraves.jpg.4      100%[===================>] 165.39K   685KB/s    in 0.2s    

2021-12-28 20:08:16 (685 KB/s) - ‘guaraves.jpg.4’ saved [169356/169356]

--2021-12-28 20:08:16--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2021-12-28 20:08:16--
Total wall clock time: 0.7s
Downloaded: 1 files, 165K in 0.2s (685 KB/s)


In [ ]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(20, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

# Processamentos na imagem (enquadramento da tela)

In [ ]:
img = cv2.imread('guaraves.jpg')
#dim = (int(img.shape[0] * 130 / 100),int(img.shape[1] * 130 / 100))
#img = cv2.resize(img,dim, interpolation = cv2.INTER_AREA)
original = img.copy()
#mostrar(img)

In [ ]:
(H, W) = img.shape[:2]

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (3, 3), 0)

edged = cv2.Canny(blur, 60, 160)

#mostrar(edged)

# Detecção de contornos na imagem

In [ ]:
def encontrar_contornos(img):
  conts = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sorted(conts, key = cv2.contourArea, reverse = True)[:6]
  return conts

conts = encontrar_contornos(edged.copy())

## Localizando o maior contorno



In [ ]:
for c in conts:
  perimetro = cv2.arcLength(c, True)
  aproximacao = cv2.approxPolyDP(c, 0.02 * perimetro, True)
  if len(aproximacao) == 4:
    maior = aproximacao
    break

cv2.drawContours(img, maior, -1, (120, 255, 0), 28)
cv2.drawContours(img, [maior], -1, (120, 255, 0), 2)
#mostrar(img)

array([[[  9,  15,  10],
        [  7,  13,   8],
        [  6,  12,   7],
        ...,
        [ 85,  99,  93],
        [ 87, 101,  95],
        [ 87, 101,  95]],

       [[  9,  15,  10],
        [  7,  13,   8],
        [  6,  12,   7],
        ...,
        [ 90, 104,  98],
        [ 91, 105,  99],
        [ 92, 106, 100]],

       [[  9,  15,  10],
        [  7,  13,   8],
        [  6,  12,   7],
        ...,
        [ 95, 109, 103],
        [ 97, 111, 105],
        [ 98, 112, 106]],

       ...,

       [[  5,   8,   6],
        [  5,   8,   6],
        [  5,   8,   6],
        ...,
        [ 23,  19,  14],
        [ 23,  19,  14],
        [ 24,  20,  15]],

       [[  5,   8,   6],
        [  5,   8,   6],
        [  5,   8,   6],
        ...,
        [ 22,  18,  13],
        [ 23,  19,  14],
        [ 23,  19,  14]],

       [[  5,   8,   6],
        [  5,   8,   6],
        [  5,   8,   6],
        ...,
        [ 22,  18,  13],
        [ 22,  18,  13],
        [ 22,  18,  13]]

##Transformação geométrica

In [ ]:
def ordenar_pontos(pontos):
  pontos = pontos.reshape((4,2))
  pontos_novos = np.zeros((4, 1, 2), dtype=np.int32)

  add = pontos.sum(1)
  pontos_novos[0] = pontos[np.argmin(add)]
  pontos_novos[2] = pontos[np.argmax(add)]

  dif = np.diff(pontos, axis = 1)
  pontos_novos[1] = pontos[np.argmin(dif)]
  pontos_novos[3] = pontos[np.argmax(dif)]

  return pontos_novos

pontos_maior = ordenar_pontos(maior)
#print(pontos_maior)

Obtenção da matriz de transformação

In [ ]:
pts1 = np.float32(pontos_maior)

In [ ]:
pts2 = np.float32([[0,0], [W, 0], [W, H], [0, H]])

In [ ]:
matriz = cv2.getPerspectiveTransform(pts1, pts2)
#matriz

Transformação de perspectiva

In [ ]:
transform = cv2.warpPerspective(original, matriz, (W, H))

#mostrar(transform) 

(função para binarizar a imagem com o opencv)

In [ ]:
def preprocessamento(img):
  maior = cv2.resize(img, None, fx=3.4, fy=3.4, interpolation=cv2.INTER_CUBIC)
  valor, otsu = cv2.threshold(maior, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  return otsu



#Reconhecimento de caracteres

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract 
import pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!mkdir tessdata
!#wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true

!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/raw/4.00/por.traineddata

mkdir: cannot create directory ‘tessdata’: File exists
--2021-12-28 20:08:27--  https://github.com/tesseract-ocr/tessdata/raw/4.00/por.traineddata
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/4.00/por.traineddata [following]
--2021-12-28 20:08:27--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/4.00/por.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21457953 (20M) [application/octet-stream]
Saving to: ‘./tessdata/por.traineddata’

./tessdata/por.trai 100%[===================>]  20.46M  --.-KB/s    in 0.1s    

2021-12-28 20:08:28 (151 MB/s) - ‘./

###Dectecção Data,Hora,local

In [ ]:
#carregando imagem
img_data_hora = original.copy()
#mostrar(img_data_hora)

In [ ]:
#modificando pontos
pontos_maior[0][0][1] -= pontos_maior[0][0][1] * (30/100)
pontos_maior[1][0][1] -= pontos_maior[1][0][1] * (30/100)

In [ ]:
#formando matiz trsformação
pts1 = np.float32(pontos_maior)
matriz = cv2.getPerspectiveTransform(pts1, pts2)

In [ ]:
#transformando a pespectiva
transform_data_hora = cv2.warpPerspective(img_data_hora, matriz, (W, H))

#mostrar(transform_data_hora) 

In [ ]:
#calculando pata fechar no cabeçario
x_data = int (transform_data_hora.shape[1] - (transform_data_hora.shape[1] * (40/100)))
y_data = int (transform_data_hora.shape[0] - (transform_data_hora.shape[0] * (85/100)))

In [ ]:
#fechando no cabeçario
transform_data_hora = transform_data_hora[:y_data,:x_data]
#mostrar(transform_data_hora)

In [ ]:
#Localizando as informações

#Local
x_local = int ((640/640) * transform_data_hora.shape[1])
y_local = int ((65/120) * transform_data_hora.shape[0])
local = transform_data_hora[:y_local,:x_local]

#data e hora
x1_data = int ((320/640) * transform_data_hora.shape[1])
x2_data = int ((640/640) * transform_data_hora.shape[1])
y1_data = int ((60/120) * transform_data_hora.shape[0])
y2_data = int ((120/120) * transform_data_hora.shape[0])
data_hora = transform_data_hora[y1_data:y2_data,x1_data:x2_data]

In [ ]:
#leituras de local,data e hora
custom_oem = r'-c tessedit_char_whitelist=abcdefghijklmnopqrstuvwxyz --psm 6'
leitura_local = pytesseract.image_to_string(local,config=custom_oem)
leitura_data_hora = pytesseract.image_to_string(data_hora)

In [ ]:
#filtrando leituras

leitura_local = leitura_local.split(' ',1)
leitura_local = leitura_local[0]
print(leitura_local)

Camaratuba


In [ ]:
#filtrando leituras

leitura_data_filtrada = ''
leitura_hora_filtrada = ''

#data e hora
leitura_data_hora = leitura_data_hora.split(' ',1)

#data
leitura_data = leitura_data_hora[0]
for i in leitura_data:
  if i == '.':
    leitura_data_filtrada = leitura_data_filtrada + '/'
  else:
    leitura_data_filtrada = leitura_data_filtrada + i

#hora
leitura_hora = leitura_data_hora[1]
for f in leitura_hora:
      if (f.isdigit() == True):
        leitura_hora_filtrada = leitura_hora_filtrada + f
      elif f == ':':
        leitura_hora_filtrada = leitura_hora_filtrada + f

#print(leitura_data_filtrada)
#print(leitura_hora_filtrada)

iniciando lista de caracteres lidos

In [ ]:
line_data_csv = []

In [ ]:
#adicionando local, data e hora no array do csv

line_data_csv.append(leitura_local)
line_data_csv.append(leitura_data_filtrada)
line_data_csv.append(leitura_hora_filtrada)

###Detecção de informações dos blocos

In [ ]:
x1 = int((85/1067) * transform.shape[1])
x1_add = int((260/1067) * transform.shape[1])

x2 = int((230/1067) * transform.shape[1])
x2_add = int((250/1067) * transform.shape[1])

y1 = int((110/800) * transform.shape[0])
y1_add = int((150/800) * transform.shape[0])

y2 = int((160/800) * transform.shape[0])
y2_add = int((160/800) * transform.shape[0])

for i in range(4):
  x1 = int((85/1067) * transform.shape[1])
  x2 = int((230/1067) * transform.shape[1])
  if i > 0:
    y1 = y1 + y1_add
    y2 = y2 + y2_add
  
  for j in range(4):
    #if i == 0 and j == 3:
     # break
    if i == 3 and j == 3:
      break
    if j > 0:
      x1 = x1 + x1_add
      x2 = x2 + x2_add

    janela=transform[y1:y2, x1:x2]

    janelaBW=cv2.cvtColor(janela, cv2.COLOR_BGR2GRAY)
    janelaBW = np.around((janelaBW-48)*1.4)
    janelaBW = janelaBW.astype(np.uint8)

    janela_processada = preprocessamento(janelaBW)

    #mostrar(janela_processada)

    leitura = pytesseract.image_to_string(janela_processada)

    #print(leitura)

    #filtro de caracteres
    leitura_filtrada = ''

    for f in leitura:
      if (f.isdigit() == True):
        leitura_filtrada = leitura_filtrada + f
      elif f == '-':
        leitura_filtrada = leitura_filtrada + f
      elif f == ',':
        leitura_filtrada = leitura_filtrada + '.'
      elif f == ' ':
        pass
      elif f == '\n':
        pass
      else:
        break
    line_data_csv.append(leitura_filtrada)

#print(line_data_csv)

###Guardando dados no CVS

In [ ]:
#cria o arquivo
arquivo_csv = open('Projeto_guaraves.csv', 'a', newline='')

#cria o objeto de gravação
w = csv.writer(arquivo_csv)

#grava linha
w.writerow(line_data_csv)

#fecha o arquivo
arquivo_csv.close()

Acessando o CSV

In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_csv('Projeto_guaraves.csv')
dados

,Local,Data,Horário,Temperatura do exterior(°C),Sensor de temperatura 4(°C),Sensor 1 de humidade (%),Estado do túnel,Sensor de temperatura 1 (°C),Sensor de temperatura 5 (°C),Sensor 2 de humidade (%),Pedido da refrigeração do túnel (%),Sensor de temperatura 2 (°C),Temperatura (°C),Sensor de pressão (Pa),Pedido de pressão estática (Pa),Sensor de temperatura 3 (°C),Humidade (%),Velocidade do ar (m/s)
0,Camaratuba,01/10/2021,11:32,-99.8,25.0,77,NaN,28.2,27.1,88,100,28.1,10.0,53.4,40.0,27.6,85,2.9
1,Camaratuba,01/10/2021,11:32,-99.8,25.0,77,NaN,28.2,27.1,88,100,28.1,10.0,53.4,40.0,27.6,85,2.9
2,Camaratuba,01/10/2021,11:32,-99.8,25.0,77,NaN,28.2,27.1,88,100,28.1,10.0,53.4,40.0,27.6,85,2.9
3,Camaratuba,01/10/2021,11:32,-99.8,25.0,77,NaN,28.2,27.1,88,100,28.1,10.0,53.4,40.0,27.6,85,2.9
4,Camaratuba,01/10/2021,11:32,-99.8,25.0,77,NaN,28.2,27.1,88,100,28.1,10.0,53.4,40.0,27.6,85,2.9
5,Camaratuba,01/10/2021,11:32,-99.8,25.0,77,NaN,28.2,27.1,88,100,28.1,10.0,53.4,40.0,27.6,85,2.9
